# Problem: We want to develop a model for minimizing the total expected cost of building a street lamp over a five-year guarantee period, assuming that each component needed to assemble an individual street lamp fails at most once over five years.

# Mathematical Formulation

Decision variables:

$ x_{i,j} $ = whether or not to purchase component $ i $ from supplier $ j $ 

where $ i = 1,...,10, j = 1,...5 $

Model parameters:

$ C_{i,j} $ = total cost of a given component $ i $ sourced from supplier $ j $

$ U_{i,j} $ = UL-certification of a given component $ i $ sourced from supplier $ j $ (binary)

$ r_{i,j} $ = defect rate of a given component $ i $ sourced from supplier $ j $

Objective function:

Minimize

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} x_{i,j} C_{i,j} (1 - r_{i,j}) + \sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} x_{i,j} 2*C_{i,j} (r_{i,j})  $

This can be simplified to:

Minimize

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} x_{i,j} C_{i,j} (1 + r_{i,j}) $

Constraints:

$ x_{i,j} \in {0,1} $ for all $ i = 1,...,10 $ for all $ j = 1,...,5 $ (x is binary variable)

One of each component number has to be selected for each light:

$\sum \limits _{j=1} ^{5} x_{i,j} = 1 $ for all $ i = 1,...,10 $

At least 7 of the 10 components must be UL-certified:

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} U_{i,j} x_{i,j} ≥ 7 $

Compatibility constraints:

$ x_{1,3} + x_{2,3} ≤ 1 $

$ x_{2,1} = x_{4,2} $

Budget:

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} x_{i,j} C_{i,j} (1 + r_{i,j}) ≤ 5400 $

# Solution

In [ ]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

In [ ]:
# Import data

df = pd.read_csv('/Users/hiradnourbakhsh/Desktop/MGSC 695_R/Assignment 2/APLO Raw Data.csv')
df

,Component #,Supplier,Defect Rate,UL Certified,Cost (NT$),Extra Pre-Assembly Cost (NT$)
0,#1,Sup11,0.037,1,1380.00,0
1,#1,Sup12,0.017,1,1668.00,0
2,#1,Sup13,0.015,1,1464.00,0
3,#1,Sup14,0.018,1,1620.00,0
4,#2,Sup21,0.043,1,278.40,0
5,#2,Sup22,0.013,0,289.20,0
6,#2,Sup23,0.060,1,252.00,0
7,#2,Sup24,0.026,1,264.00,0
8,#2,Sup25,0.024,1,270.00,0
9,#3,Sup31,0.034,1,224.00,710


In [ ]:
# Define model

model = gp.Model('APLO')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-02


## Preparing NumPy arrays

## I'm inserting dummy variables here so that all NumPy arrays have the same shape. This is for the ease of my own analysis. The dummy variables will be set to 0 below to ensure that they do not conflict with the optimal solution.

In [ ]:
sup_arr = df['Supplier'].to_numpy()
sup_arr = np.insert(sup_arr, 4, 'Sup15')
sup_arr = np.insert(sup_arr, 14, 'Sup35')
sup_arr = np.insert(sup_arr, 24, 'Sup55')
sup_arr = np.insert(sup_arr, 29, 'Sup65')
sup_arr = np.insert(sup_arr, 34, 'Sup75')
sup_arr = np.append(sup_arr, 'Sup104')
sup_arr = np.append(sup_arr, 'Sup105')
sup_arr

array(['Sup11', 'Sup12', 'Sup13', 'Sup14', 'Sup15', 'Sup21', 'Sup22',
       'Sup23', 'Sup24', 'Sup25', 'Sup31', 'Sup32', 'Sup33', 'Sup34',
       'Sup35', 'Sup41', 'Sup42', 'Sup43', 'Sup44', 'Sup45', 'Sup51',
       'Sup52', 'Sup53', 'Sup54', 'Sup55', 'Sup61', 'Sup62', 'Sup63',
       'Sup64', 'Sup65', 'Sup71', 'Sup72', 'Sup73', 'Sup74', 'Sup75',
       'Sup81', 'Sup82', 'Sup83', 'Sup84', 'Sup85', 'Sup91', 'Sup92',
       'Sup93', 'Sup94', 'Sup95', 'Sup101', 'Sup102', 'Sup103', 'Sup104',
       'Sup105'], dtype=object)

In [ ]:
def_arr = df['Defect Rate'].to_numpy()
def_arr = np.insert(def_arr, 4, 0)
def_arr = np.insert(def_arr, 14, 0)
def_arr = np.insert(def_arr, 24, 0)
def_arr = np.insert(def_arr, 29, 0)
def_arr = np.insert(def_arr, 34, 0)
def_arr = np.append(def_arr, 0)
def_arr = np.append(def_arr, 0)
def_arr

array([0.037, 0.017, 0.015, 0.018, 0.   , 0.043, 0.013, 0.06 , 0.026,
       0.024, 0.034, 0.1  , 0.011, 0.091, 0.   , 0.031, 0.012, 0.021,
       0.02 , 0.016, 0.068, 0.085, 0.075, 0.067, 0.   , 0.014, 0.014,
       0.061, 0.02 , 0.   , 0.016, 0.013, 0.03 , 0.057, 0.   , 0.035,
       0.058, 0.063, 0.046, 0.014, 0.019, 0.029, 0.059, 0.091, 0.035,
       0.077, 0.031, 0.038, 0.   , 0.   ])

In [ ]:
UL_arr = df['UL Certified'].to_numpy()
UL_arr = np.insert(UL_arr, 4, 0)
UL_arr = np.insert(UL_arr, 14, 0)
UL_arr = np.insert(UL_arr, 24, 0)
UL_arr = np.insert(UL_arr, 29, 0)
UL_arr = np.insert(UL_arr, 34, 0)
UL_arr = np.append(UL_arr, 0)
UL_arr = np.append(UL_arr, 0)
UL_arr

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0])

In [ ]:
cost_arr = df['Cost (NT$)'].to_numpy()
cost_arr = np.insert(cost_arr, 4, 0)
cost_arr = np.insert(cost_arr, 14, 0)
cost_arr = np.insert(cost_arr, 24, 0)
cost_arr = np.insert(cost_arr, 29, 0)
cost_arr = np.insert(cost_arr, 34, 0)
cost_arr = np.append(cost_arr, 0)
cost_arr = np.append(cost_arr, 0)
cost_arr

array([1380.  , 1668.  , 1464.  , 1620.  ,    0.  ,  278.4 ,  289.2 ,
        252.  ,  264.  ,  270.  ,  224.  ,  212.  ,  236.  ,  219.6 ,
          0.  ,  271.  ,  278.  ,  257.  ,  292.  ,  299.  ,  213.  ,
        221.25,  229.5 ,  218.  ,    0.  ,  684.  ,  596.  ,  460.  ,
        560.  ,    0.  ,  570.  ,  620.  ,  540.  ,  500.  ,    0.  ,
        275.  ,  257.  ,  260.  ,  300.  ,  310.  ,  420.  ,  404.  ,
        388.  ,  388.  ,  412.  ,  203.  ,  215.  ,  212.  ,    0.  ,
          0.  ])

In [ ]:
extra_arr = df['Extra Pre-Assembly Cost (NT$)'].to_numpy()
extra_arr = np.insert(extra_arr, 4, 0)
extra_arr = np.insert(extra_arr, 14, 0)
extra_arr = np.insert(extra_arr, 24, 0)
extra_arr = np.insert(extra_arr, 29, 0)
extra_arr = np.insert(extra_arr, 34, 0)
extra_arr = np.append(extra_arr, 0)
extra_arr = np.append(extra_arr, 0)
extra_arr

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 710, 587, 647,
       680,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0, 109, 107,  88, 131,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

## Reshaping NumPy Arrays

In [ ]:
sup_arr = sup_arr.reshape(10, 5)
def_arr = def_arr.reshape(10, 5)
UL_arr = UL_arr.reshape(10, 5)
cost_arr = cost_arr.reshape(10, 5)
extra_arr = extra_arr.reshape(10, 5)

In [ ]:
extra_arr

array([[  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [710, 587, 647, 680,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [109, 107,  88, 131,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0]])

In [ ]:
cost_arr

array([[1380.  , 1668.  , 1464.  , 1620.  ,    0.  ],
       [ 278.4 ,  289.2 ,  252.  ,  264.  ,  270.  ],
       [ 224.  ,  212.  ,  236.  ,  219.6 ,    0.  ],
       [ 271.  ,  278.  ,  257.  ,  292.  ,  299.  ],
       [ 213.  ,  221.25,  229.5 ,  218.  ,    0.  ],
       [ 684.  ,  596.  ,  460.  ,  560.  ,    0.  ],
       [ 570.  ,  620.  ,  540.  ,  500.  ,    0.  ],
       [ 275.  ,  257.  ,  260.  ,  300.  ,  310.  ],
       [ 420.  ,  404.  ,  388.  ,  388.  ,  412.  ],
       [ 203.  ,  215.  ,  212.  ,    0.  ,    0.  ]])

In [ ]:
C = cost_arr + extra_arr
C

array([[1380.  , 1668.  , 1464.  , 1620.  ,    0.  ],
       [ 278.4 ,  289.2 ,  252.  ,  264.  ,  270.  ],
       [ 934.  ,  799.  ,  883.  ,  899.6 ,    0.  ],
       [ 271.  ,  278.  ,  257.  ,  292.  ,  299.  ],
       [ 213.  ,  221.25,  229.5 ,  218.  ,    0.  ],
       [ 684.  ,  596.  ,  460.  ,  560.  ,    0.  ],
       [ 679.  ,  727.  ,  628.  ,  631.  ,    0.  ],
       [ 275.  ,  257.  ,  260.  ,  300.  ,  310.  ],
       [ 420.  ,  404.  ,  388.  ,  388.  ,  412.  ],
       [ 203.  ,  215.  ,  212.  ,    0.  ,    0.  ]])

In [ ]:
r = def_arr
r

array([[0.037, 0.017, 0.015, 0.018, 0.   ],
       [0.043, 0.013, 0.06 , 0.026, 0.024],
       [0.034, 0.1  , 0.011, 0.091, 0.   ],
       [0.031, 0.012, 0.021, 0.02 , 0.016],
       [0.068, 0.085, 0.075, 0.067, 0.   ],
       [0.014, 0.014, 0.061, 0.02 , 0.   ],
       [0.016, 0.013, 0.03 , 0.057, 0.   ],
       [0.035, 0.058, 0.063, 0.046, 0.014],
       [0.019, 0.029, 0.059, 0.091, 0.035],
       [0.077, 0.031, 0.038, 0.   , 0.   ]])

In [ ]:
U = UL_arr
U

array([[1, 1, 1, 1, 0],
       [1, 0, 1, 1, 1],
       [1, 0, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [0, 0, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1],
       [1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0]])

In [ ]:
# Set model parameters

components = [1,2,3,4,5,6,7,8,9,10]
suppliers = [1,2,3,4,5]

In [ ]:
l = len(components)
k = len(suppliers)
I = range(l)
J = range(k)

In [ ]:
# Define variables

x = model.addMVar((l,k), vtype = GRB.BINARY, name = ['x_' + str(i) + '_' + str(j) for i in I for j in J])



In [ ]:
# setObjective

exp = sum(x[i,j]*C[i,j] * (1 + r[i,j]) for i in I for j in J)
model.setObjective(exp, GRB.MINIMIZE)

In [ ]:
# Define constraints

# One of each component number has to be selected (need all 10 components to build a light)

for i in I:
    model.addConstr(sum(x[i,j] for j in J) == 1)

In [ ]:
# Define constraints

# at least 7 of the 10 components must be UL-certified

model.addConstr(sum(U[i,j]*x[i,j] for i in I for j in J) >= 7)


<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Define constraints

# Compatibility

model.addConstr(x[0,2] + x[1,2] <= 1)

model.addConstr(x[1,0] == x[3,1])

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Define constraints

# Budget cap

model.addConstr(exp <= 5400)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Define constraints
# Dummy variables

# Sup15, Sup35, Sup55, Sup65, Sup75, Sup104, Sup105 must be 0 (they do not actually exist)

model.addConstr(x[0,4] == 0)
model.addConstr(x[2,4] == 0)
model.addConstr(x[4,4] == 0)
model.addConstr(x[5,4] == 0)
model.addConstr(x[6,4] == 0)
model.addConstr(x[9,3] == 0)
model.addConstr(x[9,4] == 0)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# optimize

model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 21 rows, 50 columns and 135 nonzeros
Model fingerprint: 0x8cbffdd9
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 5333.3350000
Presolve removed 21 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.10 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 5117.1 5333.33 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.117103000000e+03, best bound 5.117103000000e+03, gap 0.0000%


In [ ]:
model.objVal

5117.103

# Solution: The minimal cost is $5117.103.

# Optimal allocation decisions:

In [ ]:
model.printAttr('X')


    Variable            X 
-------------------------
       x_0_0            1 
       x_1_2            1 
       x_2_2            1 
       x_3_2            1 
       x_4_0            1 
       x_5_2            1 
       x_6_2            1 
       x_7_1            1 
       x_8_2            1 
       x_9_0            1 


# We will now develop a non-linear model that maximizes the probability that no components fail over the five-year guarantee period, subject to budget constraints.

# Mathematical Formulation

Decision variables:

$ x_{i,j} $ = whether or not to purchase component $ i $ from supplier $ j $ 

where $ i = 1,...,10, j = 1,...5 $

Model parameters:

$ C_{i,j} $ = total cost of a given component $ i $ sourced from supplier $ j $

$ U_{i,j} $ = UL-certification of a given component $ i $ sourced from supplier $ j $ (binary)

$ r_{i,j} $ = defect rate of a given component $ i $ sourced from supplier $ j $

Objective function:

Minimize

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} r_{i,j} x_{i,j} $

Constraints:

$ x_{i,j} \in {0,1} $ for all $ i = 1,...,10 $ for all $ j = 1,...,5 $ (x is binary variable)

One of each component number has to be selected for each light:

$\sum \limits _{j=1} ^{5} x_{i,j} = 1 $ for all $ i = 1,...,10 $

At least 7 of the 10 components must be UL-certified:

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} U_{i,j} x_{i,j} ≥ 7 $

Compatibility constraints:

$ x_{1,3} + x_{2,3} ≤ 1 $

$ x_{2,1} = x_{4,2} $

Budget:

$\sum \limits _{i=1} ^{10}\sum \limits _{j=1} ^{5} x_{i,j} C_{i,j} (1 + r_{i,j}) ≤ 5400 $

This will be more difficult to implement because when we eliminate the assumption that each component fails at most once every five years, we increase the range of values the optimal solution can fall in.

# Solution

In [ ]:
# Define model

model = gp.Model('APLO2')

In [ ]:
# Define variables

x = model.addMVar((l,k), vtype = GRB.BINARY, name = ['x_' + str(i) + '_' + str(j) for i in I for j in J])

In [ ]:
# setObjective

exp = sum(r[i,j]*x[i,j] for i in I for j in J)
model.setObjective(exp, GRB.MINIMIZE)

In [ ]:
# add constraints


# One of each component number has to be selected (need all 10 components to build a light)

for i in I:
    model.addConstr(sum(x[i,j] for j in J) == 1)

# at least 7 of the 10 components must be UL-certified

model.addConstr(sum(U[i,j]*x[i,j] for i in I for j in J) >= 7)

# Compatibility

model.addConstr(x[0,2] + x[1,2] <= 1)

model.addConstr(x[1,0] == x[3,1])

# Budget cap

model.addConstr(sum(x[i,j]*C[i,j] * (1 + r[i,j]) for i in I for j in J) <= 5400)

# Dummy variables

# Sup15, Sup35, Sup55, Sup65, Sup75, Sup104, Sup105 must be 0

model.addConstr(x[0,4] == 0)
model.addConstr(x[2,4] == 0)
model.addConstr(x[4,4] == 0)
model.addConstr(x[5,4] == 0)
model.addConstr(x[6,4] == 0)
model.addConstr(x[9,3] == 0)
model.addConstr(x[9,4] == 0)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# optimize

model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 21 rows, 50 columns and 135 nonzeros
Model fingerprint: 0xd2b12157
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 0.4050000
Presolve removed 10 rows and 29 columns
Presolve time: 0.00s
Presolved: 11 rows, 21 columns, 48 nonzeros
Found heuristic solution: objective 0.2550000
Variable types: 0 continuous, 21 integer (21 binary)
Found heuristic solution: objective 0.2470000

Root relaxation: objective 2.254947e-01, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.22549    

In [ ]:
model.objVal

0.22699999999999998

# The optimal value of our objective function is 0.227.

In [ ]:
C[0,2] + C[1,1] + C[2,2] + C[3,2] + C[4,3] + C[5,3] + C[6,0] + C[7,4] + C[8,0] + C[9,1]

5295.2

# The cost of this solution is $5295.2.

# Optimal allocation decisions for second solution:

In [ ]:
model.printAttr('X')


    Variable            X 
-------------------------
       x_0_2            1 
       x_1_1            1 
       x_2_2            1 
       x_3_2            1 
       x_4_3            1 
       x_5_3            1 
       x_6_0            1 
       x_7_4            1 
       x_8_0            1 
       x_9_1            1 
